## 2 Supervision Object Counting 

<img src="resource/rf-supervision-banner.png" width="700px"><br><br>
- Computer Vision made easy and resusable.
    - Whether you need to load your dataset from your hard drive, 
    - draw detections on an image or video, 
    - or count how many detections are in a zone. 
    - Supervision was designed to be model agnostic. 
    - Just plug in any classification, detection, or segmentation model. 
- Website : https://supervision.roboflow.com/latest/
- Github : https://github.com/roboflow/supervision

### 2.1 Install Supervision

- Since Supervision working in `python3.8`, we need to create new conda environment with name `BelajarSuperVision` using that python version
- Open `Anaconda prompt`
- create new environment `BelajarSuperVision`
    ```
    conda create --name BelajarSuperVision python=3.8
    ```
- activate environment
    ```
    conda activate BelajarSuperVision
    ```
- run to install supervision & ultralytics
    ```
    pip install ipykernel
    pip install supervision
    pip install ultralytics
    pip install onnx --user
    pip install onnxruntime
    ```
- Close VS Code, then reopen it
- Open `6.2 supervision-object-counting.ipynb`
- Choose `BelajarSuperVision` as python environment<br>
<img src="resource/sv-image.png" width="300px">

In [1]:
# check if supervision was installed, required version >= 0.16.0
import cv2
import numpy as np
from ultralytics import YOLO
import supervision as sv

print("sv", sv.__version__)


sv 0.23.0


### 2.2 Object Counting Polygon Zone using Supervision 

#### 2.2.1 Singgle Polygon Zone

In [2]:
# paste the polygon point here!
polygon = np.array([[421, 390], [524, 470], [190, 472], [241, 398]])# CHANGE TO YOUR OWN POLYGON

In [3]:
# create supervision PolygonZone for the given polygon point
zone = sv.PolygonZone(polygon=polygon)

# create Supervision BoxAnnotator() & label_annotator()---> similar to utils.py > postprocessing_onnx()
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# create Supervision PolygonZoneAnnotator() ---> similar to utils.py > draw_object_count()
zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.WHITE, thickness=2)

In [5]:
# load Yolo V8 ONNX using Ultralytics Yolo
model = "model/yolov8s.onnx"
model = YOLO(model, task='detect')

# load video mall.mp4
cap = cv2.VideoCapture("mall.mp4")
           
# iterate for each frame in video
while cap.isOpened():
    # get image on each frame
    ret, frame = cap.read()
    if not ret:
        break

    # do forward pass (inferencing) yolo v8 onnx
    results = model(frame, imgsz=320)[0]

    # do postprocess detection result
    detections = sv.Detections.from_ultralytics(results)
    detections = detections[detections.class_id == 0] # filter only class_id = 0 --> 'person'
    zone.trigger(detections=detections)

    # draw bounding box, label 
    box_labels = [
        f"{class_name} {confidence:.2f}"
        for class_name, confidence
        in zip(detections['class_name'], detections.confidence)
    ]
    frame = box_annotator.annotate(scene=frame, detections=detections)
    frame = label_annotator.annotate(scene=frame, detections=detections, labels=box_labels)

    # draw polygon zone and object count label
    object_count_label = f"count {zone_annotator.zone.current_count}"
    frame = zone_annotator.annotate(scene=frame, label=object_count_label)

    # show result
    cv2.imshow('Frame',frame)

    # wait 1ms per frame and close using 'q'
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Loading model\yolov8s.onnx for ONNX Runtime inference...

0: 320x320 1 person, 1 chair, 51.0ms
Speed: 2.0ms preprocess, 51.0ms inference, 4.0ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 1 person, 1 chair, 51.1ms
Speed: 4.3ms preprocess, 51.1ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 1 person, 1 chair, 53.7ms
Speed: 2.0ms preprocess, 53.7ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 1 person, 1 chair, 61.6ms
Speed: 1.0ms preprocess, 61.6ms inference, 4.6ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 1 person, 1 chair, 118.0ms
Speed: 3.2ms preprocess, 118.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 1 person, 1 chair, 50.5ms
Speed: 2.0ms preprocess, 50.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 1 person, 1 chair, 64.2ms
Speed: 12.6ms preprocess, 64.2ms inference, 2.1ms postprocess per image at shape (1, 3, 320, 

#### 2.2.2 Multiple Polygon Zone

In [6]:
# paste the polygon point here!
polygon1 = np.array([[421, 390], [524, 470], [190, 472], [241, 398]]) # CHANGE TO YOUR OWN POLYGON
polygon2 = np.array([[35, 684], [118, 578], [649, 569], [822, 693]])  # CHANGE TO YOUR OWN POLYGON

In [7]:
# create supervision PolygonZone for the given polygon point
zone1 = sv.PolygonZone(polygon=polygon1)
zone2 = sv.PolygonZone(polygon=polygon2)

# create Supervision BoxAnnotator() & label_annotator()---> similar to utils.py > postprocessing_onnx()
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# create Supervision PolygonZoneAnnotator() ---> similar to utils.py > draw_object_count()
zone_annotator1 = sv.PolygonZoneAnnotator(zone=zone1, color=sv.Color.WHITE, thickness=2)
zone_annotator2 = sv.PolygonZoneAnnotator(zone=zone2, color=sv.Color.WHITE, thickness=2)

In [8]:
# load Yolo V8 ONNX using Ultralytics Yolo
model = "model/yolov8s.onnx"
model = YOLO(model, task='detect')

# load video mall.mp4
cap = cv2.VideoCapture("mall.mp4")
           
# iterate for each frame in video
while cap.isOpened():
    # get image on each frame
    ret, frame = cap.read()
    if not ret:
        break

    # do forward pass (inferencing) yolo v8 onnx
    results = model(frame, imgsz=320)[0]

    # do postprocess detection result
    detections = sv.Detections.from_ultralytics(results)
    detections = detections[detections.class_id == 0] # filter only class_id = 0 --> 'person'
    zone1.trigger(detections=detections)
    zone2.trigger(detections=detections)

    # draw bounding box, label 
    box_labels = [
        f"{class_name} {confidence:.2f}"
        for class_name, confidence
        in zip(detections['class_name'], detections.confidence)
    ]
    frame = box_annotator.annotate(scene=frame, detections=detections)
    frame = label_annotator.annotate(scene=frame, detections=detections, labels=box_labels)

    # draw polygon zone and object count label
    object_count_label1 = f"count {zone_annotator1.zone.current_count}"
    frame = zone_annotator1.annotate(scene=frame, label=object_count_label1)

    object_count_label2 = f"count {zone_annotator1.zone.current_count}"
    frame = zone_annotator2.annotate(scene=frame, label=object_count_label2)

    # show result
    cv2.imshow('Frame',frame)

    # wait 1ms per frame and close using 'q'
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Loading model\yolov8s.onnx for ONNX Runtime inference...

0: 320x320 1 person, 1 chair, 66.8ms
Speed: 4.0ms preprocess, 66.8ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 1 person, 1 chair, 57.7ms
Speed: 1.4ms preprocess, 57.7ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 1 person, 1 chair, 43.2ms
Speed: 2.3ms preprocess, 43.2ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 1 person, 1 chair, 42.2ms
Speed: 2.0ms preprocess, 42.2ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 1 person, 1 chair, 45.6ms
Speed: 1.1ms preprocess, 45.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 1 person, 1 chair, 42.2ms
Speed: 1.5ms preprocess, 42.2ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 1 person, 1 chair, 37.3ms
Speed: 1.0ms preprocess, 37.3ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 320

### Cons
- Couldn't count multiple class data, all object in polygon area will be counting together as total object in polygon area

### Pros
- More faster than OpenCV DNN

<br><br><br><br>
# Source 
- https://blog.roboflow.com/how-to-count-objects-in-a-zone/